In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_dataportal.store_list_winlife (
  store_id STRING,
  proc_date DATE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
);
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_silver_dataportal.store_list_winlife
WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dataportal.store_list_winlife_v2)
""")

In [0]:
spark.sql(
f""" 
INSERT INTO {catalog_name}.udp_wcm_silver_dataportal.store_list_winlife (store_id, proc_date)
SELECT store_id, proc_date
FROM  {catalog_name}.udp_wcm_silver_dataportal.store_list_winlife_v2 WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dataportal.store_list_winlife_v2)
"""
)